In [1]:
from azureml.core import Workspace, Experiment

# 

In [2]:

# ws = Workspace.get(name="udacity-project")
# exp = Experiment(workspace=ws, name="udacity-project")
 
# print('Workspace name: ' + ws.name, 
#      'Azure region: ' + ws.location, 
#      'Subscription id: ' + ws.subscription_id, 
#      'Resource group: ' + ws.resource_group, sep = '\n')

# run = exp.start_logging()

In [2]:
subscription_id = 'ef78944e-d555-4976-84de-e8954c8a9357'
resource_group = 'Machine_Learning'
workspace_name = 'Udemy_ML'

ws = Workspace(subscription_id, resource_group, workspace_name)

ws.write_config(path='.azureml')

exp = Experiment(workspace=ws, name='udacity-project1')

In [3]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

print('Experiment name: ' + exp.name)

Workspace name: Udemy_ML
Azure region: eastus2
Subscription id: ef78944e-d555-4976-84de-e8954c8a9357
Resource group: Machine_Learning
Experiment name: udacity-project1


In [4]:
run = exp.start_logging()      

In [6]:
# from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "project-1-CPU"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice, loguniform
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling(
    {
    '--C': uniform(0.0, 1.0),
    '--max_iter': choice(10, 40, 80, 100, 120, 150)
    }
) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

script_folder = './training'

shutil.copy('./train.py', script_folder)
est = SKLearn(source_directory =script_folder,
    entry_script = "train.py",
    compute_target=cpu_cluster_name)

### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig( 
    policy = policy,
    hyperparameter_sampling=ps,
    primary_metric_name = 'Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 15,
    max_concurrent_runs = 3,
    estimator = est
)
    ### YOUR CODE HERE ###

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
hyperdrive_run.wait_for_completion(show_output = True)

In [9]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = run.get_best_run_by_primary_metric()

model = best_run.register_model(model_name='hyperdrive_model', 
    model_path='./outputs/model.joblib')
#    model_framework=Model.Framework.SCIKITLEARN,
#    model_framework_version='0.19.1')


NameError: name 'hyperdrive_run' is not defined

In [19]:
from azureml.data.dataset_factory import TabularDatasetFactory

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

file_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(path=file_path)

In [20]:
print(ds.to_pandas_dataframe())

       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no      no   no   
32949   39    housemaid  married           basic.4y       no      no   no   

         contact month day_of_week  ...  campaign  pdays  previous  \
0    

In [21]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

NameError: name 'clean_data' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###